In the previous Notebooks i've established that using a Linear regression to predict Rank is my best bet.

This method is called a pointwise ranking method. and the "learn to rank" Theory proposes more ranking methods that i'll try to explore in this norebook. 

    -Pairwise ranking 
    -Listwise ranking

Let's start with Pairwise ranking there are various methods all from Microsoft researcher Burges
RankNet, LambdaRank, LambdaRankMart

In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')
Y_train['Rank']= Y_train['TARGET'].rank()
Y_train

,ID,TARGET,Rank
0,1054,0.028313,807.0
1,2049,-0.112516,532.0
2,1924,-0.180840,427.0
3,297,-0.260356,343.0
4,1101,-0.071733,593.0
...,...,...,...
1489,459,-0.172597,440.0
1490,1674,-0.063546,609.0
1491,748,0.151797,1016.0
1492,1454,-0.640917,163.0


In [4]:
df=create_pairwise_dataset(Y_train)
print(df)


            ID1     ID2  Comparison
0        1054.0  2049.0         2.0
1        1054.0  1924.0         2.0
2        1054.0   297.0         2.0
3        1054.0  1101.0         2.0
4        1054.0  1520.0         0.0
...         ...     ...         ...
1115266  1674.0  1454.0         2.0
1115267  1674.0   359.0         2.0
1115268   748.0  1454.0         2.0
1115269   748.0   359.0         2.0
1115270  1454.0   359.0         0.0

[1115271 rows x 3 columns]


In [22]:
import numpy.random as np_random
Comp_map = {1.0:np_random.uniform(0.66,1), 0.0: np_random.uniform(0,0.33), 0.5:np_random.uniform(0.33,0.66) } 
df['Comparison']=df['Comparison'].map(Comp_map)
df

,ID1,ID2,Comparison
0,1054.0,2049.0,0.970357
1,1054.0,1924.0,0.970357
2,1054.0,297.0,0.970357
3,1054.0,1101.0,0.970357
4,1054.0,1520.0,0.191529
...,...,...,...
1115266,1674.0,1454.0,0.970357
1115267,1674.0,359.0,0.970357
1115268,748.0,1454.0,0.970357
1115269,748.0,359.0,0.970357


In [23]:
remade=reconstruct_original_dataset(df)
remade

,ID,Rank_recreated
0,0.0,455
1,2.0,778
2,3.0,364
3,4.0,810
4,5.0,589
...,...,...
1489,2142.0,1357
1490,2143.0,1314
1491,2144.0,571
1492,2145.0,434


In [24]:
sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_train['ID'].tolist())}))
sorted['ID'].tolist()==Y_train['ID'].tolist()

True

In [25]:
from scipy.stats import spearmanr
print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["Rank_recreated"], Y_train["TARGET"]).correlation)


TARGET AND reconstructed Rank : 0.9999998515590814


Predicting Pairwise ranking accurately -> predicting Ranking accurately -> high spearman score

meaning our method of creating pairwise ranking and prediction a pairwise ranking ID by ID might help determine a more accurate ranking. Now let's Get a  X_train_pairwise that matches our Y_train_Pairwise

In [6]:
X_train_pairwise=get_training_pairwise(X_train)

Country remapped


In [13]:
print(sorted(Y_train['TARGET'].tolist(),reverse=True)[:10])

[7.786577856046542, 7.138604025388376, 6.836019876868637, 6.681753850368199, 6.151133366269985, 6.094297093901249, 6.009904429692065, 5.57011518732641, 5.0164912760257, 5.015579833097478]


In [2]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

from sklearn.linear_model import LinearRegression
from utils.compute_model_performance import *

model=Model(LinearRegression,LinearRegression)
model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=100,custom_group=2,group=True)
model.Compute(iterations=1,columns='Rank_recreated',split=False)



X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 278631
error_val -0.04070281896819743
accuracy_val [0.42731426151433255, (0.47974577810059926, 137675), (0.37608826764111114, 140935)]
list length is 278631
error_train -9.958779943246224e-17
accuracy_train [0.4684331607035829, (0.5034480815423737, 141963), (0.43207112281857096, 136665)]
        ID1     ID2  Comparison     ID1     ID2  Comparison
0    1176.0  1978.0    1.595392  1176.0  1978.0         0.0
1    1176.0  1219.0    1.044442  1176.0  1219.0         0.0
2    1176.0  1853.0    0.582380  1176.0  1853.0         2.0
3    1176.0  1520.0    0.908623  1176.0  1520.0         0.0
4    1176.0   472.0    1.053905  1176.0   472.0         0.0
..      ...     ...         ...     ...     ...         ...
995  1978.0  1123.0    0.940682  1978.0  1123.0         2.0
996  1978.0   944.0    0.750364  1978.0   944.0         0.0
997  1978.0  1569.0    0.712072

In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

from sklearn.linear_model import LogisticRegression
from utils.compute_model_performance import *

model=Model(LogisticRegression,LogisticRegression,max_iter=1000000,Solver='lbfgs')
model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=100,custom_group=2,group=True)
model.Compute(iterations=1,columns='Rank_recreated',split=False)



X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 69751
error_val -0.03013576866281487
accuracy_val [0.5687086923484967, (0.5779496423794848, 34394), (0.5597194332098312, 35357)]
list length is 626640
error_train -0.010717477339461254
accuracy_train [0.6094105068300779, (0.6183586795310875, 316221), (0.60036471069614, 310383)]
        ID1     ID2  Comparison     ID1     ID2  Comparison
0      66.0  1764.0         0.0    66.0  1764.0         2.0
1      66.0  1502.0         0.0    66.0  1502.0         2.0
2      66.0     3.0         2.0    66.0     3.0         2.0
3      66.0    92.0         0.0    66.0    92.0         0.0
4      66.0  1459.0         2.0    66.0  1459.0         2.0
..      ...     ...         ...     ...     ...         ...
995  1502.0  1350.0         2.0  1502.0  1350.0         2.0
996  1502.0  1473.0         0.0  1502.0  1473.0         0.0
997  1502.0  1351.0         2.0  1502.0  

In [5]:
from mord import OrdinalRidge
from utils.compute_model_performance import *
model=Model(OrdinalRidge,OrdinalRidge)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=2,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done


/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(
/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(


list length is 69751
error_val (-0.031211022064199796, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0])
accuracy_val [0.03346188585109891, 0.04778986345753298, 0.01930073905628198, 1.0]
list length is 626640
error_train (0.0028708668453976765, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.

/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(
/home/ymachta/miniconda3/envs/qube/lib/python3.11/site-packages/sklearn/base.py:432: UserWarning: X has feature names, but OrdinalRidge was fitted without feature names
  warnings.warn(


list length is 69751
error_val (-0.04789895485369385, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0])
accuracy_val [0.03852274519361729, 0.04162912236438998, 0.03565942506034672, 1.0]
list length is 626640
error_train (-0.0047060513213328225, [2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.

In [8]:
from sklearn.ensemble import RandomForestRegressor
model=Model(RandomForestRegressor,RandomForestRegressor,depth=13,n_estimators=2)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 278631
error_val -0.018342287236437715
accuracy_val [0.4514429478414103, (0.44832411492204244, 135779), (0.45444807717888225, 142837), (0.06666666666666667, 15)]
list length is 278631
error_train -0.00021226833914946634
accuracy_train [0.7805305224472511, (0.7786982760485388, 138693), (0.7823800131492439, 139932), (0.0, 6)]
        ID1     ID2  Comparison     ID1     ID2  Comparison
0     479.0  1596.0    0.894555   479.0  1596.0         2.0
1     479.0   207.0    1.179440   479.0   207.0         2.0
2     479.0  1948.0    0.118182   479.0  1948.0         2.0
3     479.0   470.0    0.116969   479.0   470.0         2.0
4     479.0  1165.0    0.998241   479.0  1165.0         2.0
..      ...     ...         ...     ...     ...         ...
995  1596.0  2051.0    1.087723  1596.0  2051.0         2.0
996  1596.0   533.0    1.080091  1596.0   533.0       

In [2]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from utils.compute_model_performance import *
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

# model=Model(RandomForestRegressor,RandomForestRegressor,depth=3,n_estimators=10)
# model.get_data_and_order(X_train,Y_train) 
# model.Compute(iterations=1,columns='Rank_recreated',split=False)

model=Model(RandomForestClassifier,RandomForestClassifier,depth=20,n_estimators=5)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 278631
error_val -0.04282725181333018
accuracy_val [0.5548269934070509, (0.5840807095906045, 141941), (0.5244533210426588, 136689)]
list length is 278631
error_train -0.01376372334736623
accuracy_train [0.9428132548065362, (0.9514539540754405, 144193), (0.9336954095677404, 134410), (0.21428571428571427, 28)]
        ID1     ID2  Comparison     ID1     ID2  Comparison
0    1528.0   713.0         0.0  1528.0   713.0         0.0
1    1528.0  2130.0         0.0  1528.0  2130.0         2.0
2    1528.0  2065.0         0.0  1528.0  2065.0         2.0
3    1528.0  1330.0         0.0  1528.0  1330.0         2.0
4    1528.0  2129.0         0.0  1528.0  2129.0         0.0
..      ...     ...         ...     ...     ...         ...
995   713.0   209.0         2.0   713.0   209.0         2.0
996   713.0  1366.0         2.0   713.0  1366.0         0.0
997   713.

In [2]:
import xgboost as xgb
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

model=Model(xgb.XGBRegressor,xgb.XGBClassifier,depth=3,n_estimators=50)
model.get_data_and_order(X_train,Y_train) 
model.Compute(iterations=1,columns='Rank_recreated',split=False,mode=False)

X_train and Y_train same ID ? True
X_test and Y_test same ID ? True
Country remapped
Country remapped
pairwise creation done
list length is 278631
error_val -2535.754230696708


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Conclusion Pairwise prediction with "traditional" means is not accurate enough to construct a robust ranking representation


Actually let's study our required accuracy to get an acceptable validation

In [49]:
Y_train

,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,-0.260356
4,1101,-0.071733
...,...,...
1489,459,-0.172597
1490,1674,-0.063546
1491,748,0.151797
1492,1454,-0.640917


In [52]:
sub_df=select_precentage_of_df(Y_train,25)
import scipy.stats
print(len(sub_df))
print(len(df))

sub_df['TARGET']= np.random.permutation(sub_df['TARGET'].values)
# print(sub_df)
df2 = pd.merge(Y_train, sub_df, on=['ID'], how='left', suffixes=('', '_modified'))
# Replace the original 'Value' column with the modified values
df2['TARGET'] = df2['TARGET_modified'].fillna(df2['TARGET'])
df2.drop(['TARGET_modified'], axis=1, inplace=True)
df2

373
1494


,ID,TARGET
0,1054,0.028313
1,2049,-0.112516
2,1924,-0.180840
3,297,0.121495
4,1101,-0.655248
...,...,...
1489,459,-0.172597
1490,1674,-0.063546
1491,748,0.151797
1492,1454,2.560178


In [54]:
df=create_pairwise_dataset(Y_train)
df2=create_pairwise_dataset(df2)


In [55]:
print("accuract between df and df scrambled is ",accuracy(df['Comparison'],df2['Comparison']))

accuract between df and df scrambled is  [0.8461163250904937, (0.84712247157662, 562653), (0.8451426327381178, 552573), (0.2222222222222222, 45)]


In [56]:



remade=reconstruct_original_dataset(df2)
sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_train['ID'].tolist())}))
print(np.unique(sorted['Rank_recreated']))
print(pd.concat((df.reset_index(drop=True),df2.reset_index(drop=True)),axis=1))
print(pd.concat((sorted.reset_index(drop=True),Y_train.reset_index(drop=True)),axis=1))
from scipy.stats import spearmanr
print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["Rank_recreated"], Y_train["TARGET"]).correlation)



[   1    2    3 ... 1492 1493 1494]
            ID1     ID2  Comparison     ID1     ID2  Comparison
0        1054.0  2049.0         2.0  1054.0  2049.0         2.0
1        1054.0  1924.0         2.0  1054.0  1924.0         2.0
2        1054.0   297.0         2.0  1054.0   297.0         0.0
3        1054.0  1101.0         2.0  1054.0  1101.0         2.0
4        1054.0  1520.0         0.0  1054.0  1520.0         0.0
...         ...     ...         ...     ...     ...         ...
1115266  1674.0  1454.0         2.0  1674.0  1454.0         0.0
1115267  1674.0   359.0         2.0  1674.0   359.0         2.0
1115268   748.0  1454.0         2.0   748.0  1454.0         0.0
1115269   748.0   359.0         2.0   748.0   359.0         2.0
1115270  1454.0   359.0         0.0  1454.0   359.0         2.0

[1115271 rows x 6 columns]
          ID  Rank_recreated    ID    TARGET
0     1054.0             807  1054  0.028313
1     2049.0             532  2049 -0.112516
2     1924.0             427  192

In [4]:
def process(df,percentage):
    sub_df=select_precentage_of_df(df,percentage)
    print(len(sub_df))
    print(len(df))
    sub_df['Comparison']= np.random.choice([0, 2], size=len(sub_df), p=[.5,.5])
    df2 = pd.merge(df, sub_df, on=['ID1','ID2'], how='left', suffixes=('', '_modified'))
    # Replace the original 'Value' column with the modified values
    df2['Comparison'] = df2['Comparison_modified'].fillna(df2['Comparison']).astype(int)
    df2.drop(['Comparison_modified'], axis=1, inplace=True)
    
    print("accuract between df and df scrambled is ",accuracy(df2['Comparison'],df['Comparison']))
    return df2

In [5]:
def compare(df2,Y_train,df):
    remade=reconstruct_original_dataset(df2)
    sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_train['ID'].tolist())}))
    print(np.unique(sorted['Rank_recreated']))
    print(pd.concat((df.reset_index(drop=True),df2.reset_index(drop=True)),axis=1))
    print(pd.concat((sorted.reset_index(drop=True),Y_train.reset_index(drop=True)),axis=1))
    from scipy.stats import spearmanr
    print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["Rank_recreated"], Y_train["TARGET"]).correlation)

In [1]:
import pandas as pd
from scipy.stats import spearmanr
import numpy as np
from utils.Dataset_modification import *
from utils.compute_model_performance import *

X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')

In [2]:
X_train2, X_test, Y_train2, Y_test=train_test_split(X_train,Y_train, test_size=0.25)

In [3]:
df_train=create_pairwise_dataset(Y_train)
df_test=create_pairwise_dataset(Y_test)



In [7]:
df2=process(process(df_train,10),10)
print("accuracy between df and df scrambled is ",accuracy(df2['Comparison'],df_train['Comparison']))
compare(df2,Y_train,df_train)

111527
1115271
accuract between df and df scrambled is  [0.949856133621335, (0.9501043198929943, 562213), (0.9496033547131804, 553013)]
111527
1115271
accuract between df and df scrambled is  [0.9502112042723249, (0.949933544827439, 562033), (0.9504966603096557, 553195)]
accuracy between df and df scrambled is  [0.9050114277157749, (0.9049577295437851, 562213), (0.9050691394234855, 553013)]
[ 137  142  144  147  148  149  152  153  154  156  157  158  159  160
  161  162  163  164  165  167  168  169  170  171  173  174  176  177
  178  180  181  182  183  184  185  186  187  189  190  192  193  194
  196  197  199  200  202  203  204  205  206  207  208  209  210  211
  213  215  216  217  218  220  223  224  227  228  229  232  233  234
  235  236  237  238  243  244  247  248  249  251  253  255  256  257
  258  259  260  261  262  263  267  268  269  270  271  273  274  275
  276  277  278  279  280  281  282  284  285  286  287  290  291  292
  293  295  298  299  300  301  303  3

In [12]:
compare(process(df_train),Y_train,df_train)

1081812
1115271
accuract between df and df scrambled is  [0.5148479607198609, (0.5144847237612791, 562213), (0.5152573266812895, 553013), (0.022222222222222223, 45)]
[665 673 677 681 685 686 687 689 690 691 692 694 695 696 697 698 699 700
 701 702 703 704 705 706 707 708 709 710 711 712 713 714 715 716 717 718
 719 720 721 722 723 724 725 726 727 728 729 730 731 732 733 734 735 736
 737 738 739 740 741 742 743 744 745 746 747 748 749 750 751 752 753 754
 755 756 757 758 759 760 761 762 763 764 765 766 767 768 769 770 771 772
 773 774 775 776 777 778 779 780 781 782 783 784 785 786 787 788 789 790
 791 792 793 794 795 796 797 798 799 800 801 802 803 805 807 809 812 813]
            ID1     ID2  Comparison     ID1     ID2  Comparison
0        1054.0  2049.0         2.0  1054.0  2049.0           0
1        1054.0  1924.0         2.0  1054.0  1924.0           2
2        1054.0   297.0         2.0  1054.0   297.0           2
3        1054.0  1101.0         2.0  1054.0  1101.0           2
4 

In [10]:
compare(process(df_test,100),Y_test,df_test)

69751
69751
accuract between df and df scrambled is  [0.5020286447506129, (0.5007037394784049, 36235), (0.5034760554975384, 33515)]
[160 162 164 165 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 211 212 215 220]
          ID1     ID2  Comparison     ID1     ID2  Comparison
0       490.0   603.0         0.0   490.0   603.0           2
1       490.0  1176.0         0.0   490.0  1176.0           2
2       490.0  1705.0         0.0   490.0  1705.0           0
3       490.0  1253.0         0.0   490.0  1253.0           0
4       490.0  1578.0         0.0   490.0  1578.0           0
...       ...     ...         ...     ...     ...         ...
69746  1263.0   925.0         0.0  1263.0   925.0           2
69747  1263.0  1993.0         0.0  1263.0  1993.0           2
69748   929.0   925.0         0.0   929.0   925.0           2
69749   929.0  1993.0         0.0   929.

In [26]:
X_train = pd.read_csv(datasets+'X_train.csv') #datasets is defined in utils
Y_train=pd.read_csv(datasets+'Y_train.csv')
X_train2, X_test, Y_train2, Y_test=train_test_split(X_train,Y_train, test_size=0.25)
df_train=create_pairwise_dataset(Y_train)
df2=process(df_train)





836453
1115271
accuract between df and df scrambled is  [0.6250427026256399, (0.6281284851115146, 562213), (0.6219293217338471, 553013), (0.3333333333333333, 45)]


In [23]:
df_train
df2

,ID1,ID2,Comparison
0,1054.0,2049.0,2
1,1054.0,1924.0,0
2,1054.0,297.0,2
3,1054.0,1101.0,0
4,1054.0,1520.0,0
...,...,...,...
1115266,1674.0,1454.0,0
1115267,1674.0,359.0,2
1115268,748.0,1454.0,0
1115269,748.0,359.0,2


In [27]:
remade=reconstruct_original_dataset(df2)
sorted=remade.sort_values(by=['ID'], key=lambda x: x.map({k: i for i, k in enumerate(Y_test['ID'].tolist())})).head(len(Y_test))
print(sorted)
print(np.unique(sorted['Rank_recreated']))

print(pd.concat((sorted.reset_index(drop=True),Y_test.reset_index(drop=True)),axis=1))
from scipy.stats import spearmanr
print("TARGET AND reconstructed Rank :" ,spearmanr(sorted["Rank_recreated"], Y_test["TARGET"]).correlation)

          ID  Rank_recreated
1197  1718.0             565
1034  1481.0             686
1473  2123.0             783
1179  1693.0             619
1319  1886.0             954
...      ...             ...
200    289.0             933
208    297.0             684
744   1075.0             744
1400  2016.0             737
830   1188.0             848

[374 rows x 2 columns]
[520 536 539 547 548 552 557 559 560 564 565 569 570 571 574 575 576 577
 578 579 583 588 589 592 594 596 597 599 600 604 606 607 608 610 611 612
 613 617 619 620 621 622 623 628 629 631 632 633 634 635 636 637 638 639
 640 641 644 645 648 649 650 652 654 655 656 658 659 661 664 665 668 670
 672 674 675 677 678 679 680 681 684 686 687 689 690 691 694 695 696 697
 698 703 704 706 708 710 711 712 713 715 716 717 720 721 722 726 727 729
 730 731 732 733 737 741 742 744 745 749 750 751 753 754 755 756 757 758
 759 760 761 763 764 765 766 768 772 774 776 777 778 779 780 782 783 786
 787 788 790 795 796 801 802 803 805 806 807